<a href="https://colab.research.google.com/github/rgclapp007/gp211-class-notebooks/blob/main/regularization/miss1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install "sep_plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@2bffacb9fb36963339a0834c2b04a0aedff91db4"


In [ ]:
import numba
from genericSolver.pyOperator import Operator, ChainOperator
from sep_python.sep_vector import FloatVector,get_sep_vector
import numpy as np
class tconv(Operator):

  def __init__(self,mod:FloatVector,dat:FloatVector,filt:np.ndarray):
    if not isinstance(mod,FloatVector) or not isinstance(dat,FloatVector):
      raise Exception(f"Expecting float vectors got {type(mod)} and {type(dat)}")
    nm=mod.get_hyper().get_ns()
    nd=dat.get_hyper().get_ns()

    if not isinstance(filt,np.ndarray):
      raise Exception("Expecting filter to be an n-d array")

    if len(list(filt.shape))!=1:
      raise Exception("Expecting filter to be 1-D")
    
    if len(nm) !=1 or len(nd)!=1:
      raise Exception("Expecting 1-D vectors")
    
    if nd[0]!=nm[0]+filt.shape[0]-1:
      raise Exception("Expecting size of data to be len(filt)+len(mod)-1")
    
    self._filt=np.copy(filt)

    super().__init__(mod,dat)

  def forward(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      dat.zero()

    tconv_forward(mod.get_nd_array(),dat.get_nd_array(),self._filt)

  def adjoint(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      mod.zero()

    tconv_adjoint(mod.get_nd_array(),dat.get_nd_array(),self._filt)
  
@numba.njit()
def tconv_forward(mod,dat,filt):
  for imod in range(mod.shape[0]):
    for ifilt in range(filt.shape[0]):
      dat[imod+ifilt]+=filt[ifilt]*mod[imod]

@numba.njit()
def tconv_adjoint(mod,dat,filt):
  for imod in range(mod.shape[0]):
    for ifilt in range(filt.shape[0]):
      mod[imod]+=filt[ifilt]*dat[imod+ifilt]


In [ ]:
import copy
class Jop(Operator):

  def __init__(self,mod:FloatVector,dat:FloatVector,sc):
    if not mod.checkSame(data):
        raise Exception("Model and data must be the same")
    
    self._sc=copy.deepcopy(np.ravel(sc))
    if mod.get_hyper().get_n123()!=self._sc.shape[0]:
        raise Exception(f"Expecting mask {self.m.shape[0]} and model {mod.get_hyper().get_n123()} to be same size ")

    super().__init__(mod,dat)

  def forward(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      dat.zero()
    
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    d[:]+=m[:]*self._sc[:]

  def adjoint(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      mod.zero()
    d=np.ravel(dat.get_nd_array())
    m=np.ravel(mod.get_nd_array())
    m[:]+=d[:]*self._sc[:]
  


In [ ]:
class  causalInt(Operator):

  def __init__(self,mod:FloatVector,dat:FloatVector):

    if not mod.checkSame(dat):
        raise Exception("Model and data not same space")

    super().__init__(mod,dat)

  def forward(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      dat.zero()

    caus_forward(mod.get_nd_array(),dat.get_nd_array())

  def adjoint(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      mod.zero()
    caus_adjoint(mod.get_nd_array(),dat.get_nd_array())


import math  
@numba.njit()
def caus_forward(mod,dat):
  t=0
  for imod in range(mod.shape[0]):
    t=t+mod[imod]
    dat[imod]+=t

@numba.njit()
def caus_adjoint(mod,dat):
  t=0
  for imod in range(mod.shape[0]-1,-1,-1):
      t=t+dat[imod]
      mod[imod]+=t
    
class causalBoth(Operator):
    def __init__(self,model,data):
        self._op=causalInt(model,data)
        self._tmp=model.clone()
        super().__init__(model,data)
    
    def forward(self,add,model,data):
        self._op.forward(False,model,self._tmp)
        self._op.adjoint(add,data,self._tmp)

    def adjoint(self,add,model,data):
        self._op.adjoint(False,self._tmp,data)
        self._op.forward(add,self._tmp,data)
    


In [ ]:
from genericSolver.pyProblem import ProblemL2Linear,ProblemL2LinearReg
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
import holoviews as hv
from sep_plot import Dots



In [ ]:
import holoviews as hv
from sep_plot import Graph
hv.extension('bokeh','matplotlib')
fill = np.cos(np.linspace(start = -np.pi*(3/2), stop = 5*np.pi*(3/2), num = 300)).astype(np.float32)
sc=np.zeros((300))
sc[:]=1
x=(300*(np.sqrt(np.random.rand(200)))).astype(np.int32)
for i in x:
    fill[i]=0
inp=get_sep_vector(fill)
filt=np.array([1,-1.])
model=inp.clone()
start_model=model.clone()
nm=model.get_hyper().get_ns()
data_reg=get_sep_vector(ns=[nm[0]+filt.shape[0]-1])
unknown=model.clone()
known=model.clone()
un=unknown.get_nd_array()
kn=known.get_nd_array()
un[:]=1
kn[:]=0
for ival,val in enumerate(model.get_nd_array()):
    if val !=0:
      kn[ival]=1
      un[ival]=0

op=tconv(model,data_reg,filt)
prob_base=ProblemL2Linear(model,data_reg,op,grad_mask=unknown)
stop=BasicStopper(niter=nm[0])

solve_base=LCGsolver(stop)
solve_base.setDefaults(save_model=True)
solve_base.run(prob_base)

Graph(model)+Graph(prob_base.model)

In [ ]:
data_fit=start_model.clone()
tconv_op=tconv(model,data_reg,filt)

jop=Jop(model,data_fit,kn)
model.zero()
prob_reg=ProblemL2LinearReg(model,data_fit,jop,reg_op=tconv_op,epsilon=1.)
stop=BasicStopper(niter=nm[0])

solve_reg=LCGsolver(stop)
solve_reg.setDefaults(save_model=True)
solve_reg.run(prob_reg)
Graph(data_fit,width=700)+Graph(prob_reg.model,width=700)

In [ ]:
data_fit=start_model.clone()
jop=Jop(model,data_fit,kn)
ones=np.ones((300))
iop=Jop(model,model,ones)

causal=causalInt(model,model)
op_fit=ChainOperator(causal,jop)
model.zero()
op_fit.dotTest()
prob_prec=ProblemL2LinearReg(model,data_fit,op_fit,reg_op=iop,epsilon=.000000)
stop=BasicStopper(niter=100)

solve_prec=LCGsolver(stop)
solve_prec.setDefaults(save_model=True)
solve_prec.run(prob_prec)
tmp=prob_prec.model.clone()

prob_prec.op.forward(False,prob_prec.model,prob_prec.res)
causal.forward(False,prob_prec.model,tmp)
Graph(prob_prec.data)+Graph(tmp)



In [ ]:
print(mod.max(),mod.min())

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
def plot_result_reg(iter):
    return (Graph(data_fit,width=400)+Graph(solve_reg.modelSet.vecSet[iter],width=400)).cols(1)

# When run live, this cell's output should match the behavior of the GIF below
dmap = hv.DynamicMap(plot_result_reg, kdims=['iter'])
dmap.redim.range(iter=(0,len(solve_reg.modelSet.vecSet)-1))
#dmap = dmap.redim.values(Method=methods)


In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
def plot_result_prec(iter):
    tmp=prob_prec.model.clone()
    causal.forward(False,solve_prec.modelSet.vecSet[iter],tmp)
    return (Graph(data_fit,width=700)+Graph(tmp,width=700)).cols(1)

# When run live, this cell's output should match the behavior of the GIF below
dmap = hv.DynamicMap(plot_result_prec, kdims=['iter'])
dmap.redim.range(iter=(0,50))